<a href="https://colab.research.google.com/github/tqnhu2407/Linear_Regression/blob/master/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Đồ án 03: Linear Regression

## Thông tin cá nhân

### Họ tên: Trần Quỳnh Như

### MSSV: 18127266

## Các chức năng đã hoàn thành

Xây dựng mô hình đánh giá chất lượng rượu sử dụng phương pháp hồi quy tuyến tính.

    a) Sử dụng toàn bộ 11 đặc trưng đề bài cung cấp.

    b) Sử dụng duy nhất 1 đặc trưng cho kết quả tốt nhất.

## Ý tưởng thực hiện

Mô hình hồi qui tuyến tính của ta có dạng 
$$A\hat{x} = b$$
nên ta tách tập dữ liệu ra thành 2 ma trận **A** và **b**

Ở câu a) ta chỉ cần gọi hàm của numpy để tính nghiệm $\hat{x}$

Ở câu b) ta tách ma trận A ra thành 11 thuộc tính độc lập. Ở mỗi thuộc tính, ta chia dữ liệu ra thành 2 phần train và test. Sau khi tính được nghiệm $\hat{x}$ từ tập train, ta thay vào tập test để tính

$$\hat{r} = A\hat{x} - b$$

Từ đó ta tính được trung bình các độ chênh lệch của tập train so với tập test.

Thuộc tính nào có sai số trung bình nhỏ nhất thì đó là thuộc tính tốt nhất.



## Mô tả các hàm chức năng

### Tiền xử lí dữ liệu

Import các thư viện cần thiết và đọc file csv bằng thư viện pandas.

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('wine.csv', sep=';')

Gán data vào numpy array mới để thuận lợi cho việc xử lí.

Tách tập dữ liệu ra thành 2 ma trận A và b

In [2]:
wine = df.values

A = wine[:, :-1]
b = wine[:, 11]

### a) Sử dụng toàn bộ 11 đặc trưng đề bài cung cấp.

Tính nghiệm của phương trình hồi qui tuyến tính.

In [3]:
x_hat = np.linalg.pinv(A)@b
x_hat

array([ 5.92516137e-03, -1.10803754e+00, -2.63046284e-01,  1.53222831e-02,
       -1.73050274e+00,  3.80141908e-03, -3.89899869e-03,  4.33858768e+00,
       -4.58535475e-01,  7.29718662e-01,  3.08858648e-01])

### b) Sử dụng duy nhất 1 đặc trưng cho kết quả tốt nhất.

Do tập dữ liệu có **1199** dòng nên ta **duplicate dòng cuối** để **chẵn 1200 dòng**, thuận tiện cho việc tính toán ở hàm cross_validation ở dưới.

**Lưu ý: cell này chỉ được run 1 lần vì run nhiều lần sẽ append nhiều dòng.**

In [4]:
last_A = A[-1]
last_b = b[-1]

A = np.append(A, [last_A], axis=0)
b = np.append(b, [last_b], axis=0)

Chia tập rượu thành 11 thuộc tính att[0] ... att[10].

In [5]:
att = np.zeros((11, 1200, 1))

for col_idx in range(11):
    att[col_idx, :, 0] = A[:, col_idx]

Ta xây hàm cross_validation với tham số truyền vào là n_group cho phép người dùng nhập số lượng cụm data. (Số cụm nên là ước của 1200: 3, 4, 5,...)

**Giả sử n_group = 3** thì dữ liệu mỗi thuộc tính được chia làm **3 phần bằng nhau** là A (từ line 0 đến 399), B (từ line 400 đến 799) và C (từ line 800 đến 1199). Ta dùng 2 phần để train và 1 phần để test.

Ví dụ:

Train | Test

. A B | C

. A C | B

. B C | A

Ta tạo list errors chứa 11 sai số ứng với 11 thuộc tính. 

Xét mỗi thuộc tính, thực hiện chia dữ liệu ra 2 phần train-test và tính sai số của lần chia đó.

Từ đó tính được sai số của thuộc tính bằng cách: lấy trị tuyệt đối của trung bình cộng các sai số của các lần chia train-test.

Kết quả trả về của hàm là list errors.

In [6]:
def cross_validation(n_group):

    errors = []
    
    for i in range(11):

        # Errors of this attribute
        att_errors = []
        step = 1200 // n_group

        # Divide each attribute's dataset into 2 subsets train and test
        for j in range(0, 1200, step):
            
            # DELETE LINE j TO LINE j+step of this attribute's dataset
            # bcz those lines belong to the test set
            trainAtt = att[i].copy()
            trainb = b.copy()
            for k in range(0, step): # DELETE LINE j TO LINE j+step
                trainAtt = np.delete(trainAtt, j, 0)
                trainb = np.delete(trainb, j, 0)

            # The test set will go from line j to line j+step
            testAtt = att[i][j:j+step]
            testb = b[j:j+step]

            # Calculate x hat and r hat
            x_hat = np.linalg.pinv(trainAtt)@trainb
            r_hat = abs(testAtt @ x_hat - testb)
            
            # Calculate the average of this
            att_errors.append(np.average(r_hat))
        
        # This attribute's error
        errors.append(abs(np.average(att_errors)))
    
    return errors

In [7]:
errors = cross_validation(4)

Tìm sai số nhỏ nhất trong list errors.

In [8]:
min_error = min(errors)
min_error_idx = errors.index(min_error)

Danh sách các sai số của các thuộc tính

In [9]:
list_errors = list(zip(df.columns, errors))
list_errors

[('fixed acidity', 1.077375392682978),
 ('volatile acidity', 1.86559257103894),
 ('citric acid', 2.648960273112925),
 ('residual sugar', 2.013649878078203),
 ('chlorides', 2.1098784623384885),
 ('free sulfur dioxide', 2.8416967795130867),
 ('total sulfur dioxide', 3.136373010227495),
 ('density', 0.7145300013793484),
 ('pH', 0.7426240359792065),
 ('sulphates', 1.0974037951682105),
 ('alcohol', 0.5472616465482669)]

Thuộc tính tốt nhất với sai số nhỏ nhất.

In [10]:
list_errors[min_error_idx]

('alcohol', 0.5472616465482669)